In [ ]:
"""
To do:
next: add image features from CNN into autoencoder and isolation forest
1) figure out how to feature engineer with images (rotations, crops, etc.)
2) load images and metadata into autoencoder and get predictions
"""

In [21]:
# Import libraries
import os
import zipfile
import numpy as np
import pandas as pd
from io import BytesIO
import tensorflow as tf
import scipy.stats as stats
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from concurrent.futures import ThreadPoolExecutor
from keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.feature_selection import mutual_info_classif, SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance

# Loading the Data

In [2]:
# Create functions to load, process, and feature extract the images
def extract_features(img_batch):
    """
    Extracts features from a batch of images using ResNet50
    
    :param img_batch: batch of images as numpy arrays
    :return: feature vectors for the batch
    """
    # Preprocess batch images to align with ResNet50 requirements
    img_batch = np.array([preprocess_input(img) for img in img_batch])
    
    # Extract features for the entire batch
    features = resnet_model.predict(img_batch, verbose=0)
    
    # Flatten and return the features
    return features.reshape(features.shape[0], -1)

def load_image(zip_fold, file, img_size):
    """
    Loads and resizes an image from a zip folder
    
    :param zip_fold: zipped folder object
    :param file: filename for image in zip folder
    :param img_size: target size for resizing
    :return: image as a numpy array
    """
    with zip_fold.open(file) as img_file:
        img = load_img(BytesIO(img_file.read()), target_size=img_size)
        return img_to_array(img)

def get_batch_features(files, z, img_size=(224, 224), batch_size=32, num_workers=4):
    """
    Loads images and extracts feautres with batch processing and a ResNet50 model
    
    :param files: list of filenames for images
    :param z: zipped folder object
    :param img_size: target size to resize images
    :param batch_size: number of images per batch
    :param num_workers: number of threads for parallel processing
    :return: feature vectors for images in batches
    """
    
        
    # Process images in batches
    for i in range(0, len(files), batch_size):
        batch_files = files[i:i + batch_size]

        # Load and preprocess images in parallel
        with ThreadPoolExecutor(max_workers=num_workers) as executor:
            img_batch = list(executor.map(lambda file: load_image(z, file, img_size), batch_files))

        # Extract features for the batch
        img_batch = np.array(img_batch)
        features = extract_features(img_batch)
            
        yield features

In [2]:
# Load in the metadata
zip_folder = zipfile.ZipFile('anon-patient-data.zip')
skin_cancer_df = pd.read_csv(zip_folder.open('train-metadata.csv'), low_memory=False, 
                            usecols=[num for num in range(0, 43) if num not in [2, 7]], index_col='isic_id')

# Randomly undersample the non-cancerous individuals from the dataset
rus = RandomUnderSampler(random_state=42, sampling_strategy=0.31)
skin_cancer_df, targets = rus.fit(skin_cancer_df.drop('target'), skin_cancer_df['target'])
img_files = [file[6:-4] for file in zip_folder.namelist() if file.startswith('image/')]
removed_samples = rus.sample_indices_
removed_samples
"""
# Load the pre-trained ResNet50 model
resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
tf.config.list_physical_devices('GPU')

# Obtain all JPG image filenames and initialize array for image features
img_files = [file for file in zip_folder.namelist() if file.startswith('image/') and file.endswith('.jpg')]
images_features = np.zeros((len(img_files), 2048))

# Extract image features using ResNet50 and batch processsing
idx = 0
for batch_features in get_batch_features(img_files, zip_folder, img_size=(224, 224), batch_size=64, num_workers=4):
    batch_length = batch_features.shape[0]
    images_features[idx:idx + batch_length] = batch_features
    idx += batch_length
    
# Save image data into CSV to avoid unnecesary repetition of ResNet50 process
pd.DataFrame(images_features).to_csv('resnet50_features.csv', index=False, header=False)
"""

In [ ]:
# Read CSV data containing image features from ResNet50 processing commented out above
images_features = np.loadtxt('resnet50_features.csv', delimiter=',')

# Exploratory Data Analysis of Metadata by Lesion Type (Cancer vs Non-Cancer)

### How balanced is the data?

In [ ]:
# Report the number of cancerous vs non-cancerous lesions in the data
not_cancer = skin_cancer_df[skin_cancer_df['target'] == 0]
cancer = skin_cancer_df[skin_cancer_df['target'] == 1]
print(f'Out of the {len(skin_cancer_df)} lesions in our dataset, {len(not_cancer)} are not cancerous and {len(cancer)} are cancerous.')

# Visualize the results in a pie chart
fig, ax = plt.subplots()
ax.pie([len(not_cancer), len(cancer)], labels=['Not Cancer', 'Cancer'], autopct='%1.1f%%')
ax.set_title('Proportion of Cancerous vs Non-Cancerous Lesions')
plt.show()

#### The data is heavily imbalanced, with almost all available lesions being non-cancerous. This characteristic of the data is our primary motivator for utilizing anomaly detection rather than binary classification as our method for cancer detection.

### Do men and women make up different proportions of cancerous vs non-cancerous lesions?

In [ ]:
# Obtain the frequencies of each sex for cancerous and non-cancerous lesions
gender_freqs_cancer = Counter(cancer['sex'])
gender_freqs_noncancer = Counter(not_cancer['sex'])

# Visualize the frequencies
fig, ax = plt.subplots(1,2)
ax[0].pie([gender_freqs_noncancer['male'], gender_freqs_noncancer['female'], gender_freqs_noncancer[np.nan]],
       labels=['male', 'female', 'NA'], autopct='%1.1f%%')
ax[0].set_title('Non-Cancerous Patients')
ax[1].pie([gender_freqs_cancer['male'], gender_freqs_cancer['female'], gender_freqs_cancer[np.nan]],
       labels=['male', 'female', 'NA'], autopct='%1.1f%%')
ax[1].set_title('Cancerous Patients')
plt.show()

#### Men are more represented in cancerous lesions than non-cancerous lesions, which aligns with the notion that men are more likely to obtain skin cancer

### Is there a significant difference in the age distribution for cancerous vs non-cancerous patients?

In [ ]:
# Visualize the age distributions
plt.hist(cancer['age_approx'], histtype='step', color='red', density=True, label='Cancerous')
plt.hist(not_cancer['age_approx'], histtype='step', color='green', density=True, label='Non-Cancerous')
plt.legend()
plt.xlabel('Age Approximations')
plt.ylabel('Probability')
plt.title('Age Distribution of Cancerous vs Non-Cancerous Patients')
plt.show()

# Perform the Mann-Whitney U test
u_stat, p_value = stats.mannwhitneyu(cancer['age_approx'], not_cancer['age_approx'])

# Print the result
print(f'Mann-Whitney U test: U-stat = {u_stat}, p-value = {p_value}')

# Interpretation
if p_value < 0.05:
    print('There is a significant difference in the age distribution between cancerous and non-cancerous patients.')
else:
    print('There is no significant difference in the age distribution between cancerous and non-cancerous patients.')

### Summary Statistics

In [ ]:
# Define the columns to compare summary stats for (choose columns that align with the ABCD factors used for skin cancer detection)
use_cols = ['tbp_lv_symm_2axis', 'tbp_lv_area_perim_ratio', 'tbp_lv_color_std_mean', 'clin_size_long_diam_mm']

# Present summary statistics for cancerous patients
cancer[use_cols].describe()

In [ ]:
# Present summary statistics for non-cancerous patients
not_cancer[use_cols].describe()

In [ ]:
# Use the Mann-Whitney U test to determine if any of these differences are significant
for col in use_cols:
    u_stat, p_value = stats.mannwhitneyu(cancer[col], not_cancer[col])
    if p_value < 0.05:
        print(f'There is a significant difference in {col} between cancerous and non-cancerous patients.')
    else:
        print(f'There is no significant difference in {col} between cancerous and non-cancerous patients.')

### Null Values

In [ ]:
# Identify columns with null values for non-cancerous patients
not_cancer_nulls = filter(lambda item: item[1] > 0, not_cancer.isnull().sum().items())
print('Columns with null values for non-cancerous patients:')
for tup in not_cancer_nulls:
    print(f'Column: {tup[0]}, No. of Nulls: {tup[1]}, As a %: {round(tup[1]/len(cancer[tup[0]]), 2)}')
    
# Identify columns with null values for cancerous patients
cancer_nulls = filter(lambda item: item[1] > 0, cancer.isnull().sum().items())
print('\nColumns with null values for cancerous patients:')
for tup in cancer_nulls:
    print(f'Column: {tup[0]}, No. of Nulls: {tup[1]}, As a %: {round(tup[1]/len(cancer[tup[0]]), 2)}')

# Data Preprocessing

In [3]:
# Obtain the categorical (nominal) features
categorical_features = skin_cancer_df.select_dtypes(include=['object', 'category', 'string']).columns.tolist()

# Impute and encode values in categorical columns
updated_features = []
for feature in categorical_features:
    
    # Impute null values in categorical features with the mode
    skin_cancer_df[feature] = skin_cancer_df[feature].fillna(skin_cancer_df[feature].mode()[0])
    
    # Apply one-hot encoding to categorical (nominal) variables
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoded_feature = encoder.fit_transform(skin_cancer_df[[feature]])
    
    # Add the encoded columns to the dataframe
    encoded_col_names = [f"{feature}_{cat}" for cat in encoder.categories_[0]]
    encoded_feature_df = pd.DataFrame(encoded_feature, columns=encoded_col_names, index=skin_cancer_df.index)
    skin_cancer_df = pd.concat([skin_cancer_df, encoded_feature_df], axis=1)
    updated_features += encoded_col_names
    
# Remove unencoded categorical columns
skin_cancer_df = skin_cancer_df.drop(columns=categorical_features)
updated_cols = skin_cancer_df.columns
 
# Use KNN to impute null values in the numerical columns
imputer = KNNImputer(n_neighbors=5)
imputed_array = imputer.fit_transform(skin_cancer_df)
skin_cancer_df = pd.DataFrame(imputed_array, columns=updated_cols, index=skin_cancer_df.index)

# Feature Engineering

In [4]:
# choosing which features to use from feature engineering
def create_features(df):
    """
    Creates new features to help the model evaluate the ABCD factors used by dermatologists
    :param df: a dataframe to add new features to
    :return: the input dataframe with updated features
    """
    og_cols = len(df.columns)
    # A - Asymmetry, Border irregularity/bluriness, and Diameter (skin cancer diameter usually > 6 mm)
    df['diameter_ratio'] = df['tbp_lv_minorAxisMM'] / df['clin_size_long_diam_mm']
    df['area_irregularity'] = np.abs((np.pi * (df['clin_size_long_diam_mm'] / 2)**2) - (df['tbp_lv_areaMM2'])**(1/2))
    df['perimeter_irregularity'] = np.abs((np.pi * df['clin_size_long_diam_mm']) - df['tbp_lv_perimeterMM'])
    df['area_perimeter_ratio'] = df['tbp_lv_areaMM2'] / (df['tbp_lv_perimeterMM'] ** 2)
    df['large_diameter'] = [1 if val > 5.5 else 0 for val in df['clin_size_long_diam_mm']] # skin cancer diameters tend to be larger than 6 mm
    df['perimeter_to_area'] = (df['tbp_lv_perimeterMM']**2) / df['tbp_lv_areaMM2']
    df['avg_normalized_irregularity'] = (df["tbp_lv_norm_border"] + df["tbp_lv_norm_color"]) / 2
    
    # Color (variation)  
    df['hc_mean_contrast'] = ((df['tbp_lv_H'] + df['tbp_lv_Hext']) / 2) + ((df['tbp_lv_C'] + df['tbp_lv_Cext']) / 2)
    df['tbp_lv_deltaH'] = np.abs(df['tbp_lv_H'] - df['tbp_lv_Hext'])
    df['tbp_lv_deltaC'] = np.abs(df['tbp_lv_C'] + df['tbp_lv_Cext'])
    df['overall_lab_contrast'] = np.sqrt(df['tbp_lv_deltaL']**2 + df['tbp_lv_deltaA']**2 + df['tbp_lv_deltaB']**2)
    df['large_color_variance'] = [1 if val > 4 else 0 for val in df['tbp_lv_color_std_mean']]
    df['average_lab_contrast'] = (df['tbp_lv_deltaL'] + df['tbp_lv_deltaA'] + df['tbp_lv_deltaB']) / 3
    
    # Features to maximize other features
    df['lesion_location'] = np.sqrt(df['tbp_lv_x']**2 + df['tbp_lv_y']**2 + df['tbp_lv_z']**2) # l2 norm of lesion coordinates
    print(f'Created {len(df.columns) - og_cols} New Features During Feature Engineering')
    df = df.drop(['tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z'], axis=1)

    return df

# Apply feature engineering
skin_cancer_enhanced = create_features(skin_cancer_df.copy())

Created 14 New Features During Feature Engineering


In [5]:
# maybe use these features in feature engineering

def create_new_features(df):
    # Create new features
    df["color_uniformity"] = np.where(df["tbp_lv_radial_color_std_max"] == 0, 0, (df["tbp_lv_color_std_mean"] / df["tbp_lv_radial_color_std_max"]))
    df["size_age_interaction"] = df["clin_size_long_diam_mm"] * df["age_approx"]
    
    df["log_lesion_area"] = np.log(df["tbp_lv_areaMM2"] + 1)
    df["mean_hue_difference"] = (df["tbp_lv_H"] + df["tbp_lv_Hext"]) / 2

    return df

skin_cancer_enhanced = create_new_features(skin_cancer_enhanced.copy())

# Correlation Matrix

In [5]:
# Calculate the correlation matrix
og_cols = len(skin_cancer_enhanced.columns)
corr_matrix = skin_cancer_enhanced.corr().abs()

# Remove repetitions by only keeping values in upper, right triangle
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Drop features with correlations to other features greater than 0.8
threshold = 0.8
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
skin_cancer_enhanced = skin_cancer_enhanced.drop(to_drop, axis=1)
print(f'Removed {og_cols - len(skin_cancer_enhanced.columns)} Features with High Correlation to Another Feature')

Removed 31 Features with High Correlation to Another Feature


# Feature Importances

In [ ]:
# Calculate feature importances using ANOVA f-test
fs = SelectKBest(score_func=f_classif, k='all')
skin_cancer_array = fs.fit_transform(skin_cancer_enhanced.drop(['target', 'image_path'], axis=1), skin_cancer_enhanced['target'])
use_cols = skin_cancer_enhanced.drop(['target', 'image_path'], axis=1).columns

In [ ]:
# Plot the feature importances
plt.figure(figsize=(15,5))
plt.bar([use_cols[i] for i in range(len(fs.scores_))], fs.scores_)
plt.xticks(rotation='vertical')
plt.show()

# Aggregate Data

In [38]:
# Combine enhanced metadata and image features into one data set
images_features = pd.DataFrame(images_features)
skin_cancer_features = skin_cancer_enhanced.drop('target', axis=1)
skin_cancer_full = pd.concat([skin_cancer_features.reset_index(), images_features], axis=1)
skin_cancer_full.columns = skin_cancer_full.columns.astype(str)

# Isolation Forest - gives worse results than autoencoders

In [44]:
best = (0, 0, 0, None)
for ss in [x*0.01 for x in range(21, 33, 2)]:
    # Oversample the minority group to make the data more balanced
    smote = SMOTE(sampling_strategy=ss, random_state=42, n_jobs=3)
    X_resampled, y_resampled = smote.fit_resample(skin_cancer_full.drop('isic_id', axis=1), skin_cancer_df['target'])

    # Split the data - training is non-cancerous, test is on all patients to detect anomalies
    X_train = X_resampled[y_resampled == 0]
    X_test = X_resampled

    # Scale the data between 0 and 1
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Develop and train the Isolation Forest model
    c = len(y_resampled[y_resampled == 1]) / len(y_resampled[y_resampled == 0])
    for estimators in range(60, 120, 10):
        isf = IsolationForest(n_estimators=estimators, contamination=c, random_state=42)
        isf.fit(X_train)

        # Predict the targets for the test data
        preds = isf.predict(X_test)
        y_preds = [1 if p == -1 else 0 for p in preds]

        # Evaluate the models performance on testing data
        cr = classification_report(y_resampled, y_preds)
        f1_score = float(cr.split()[12])
        if f1_score > best[2]:
            best = (ss, estimators, f1_score, cr)
            print(f'\nS.S.: {ss}, Estimators: {estimators}, f1_score: {f1_score}')
            print(cr)
print('Best Hyperparameters + result:', best[:2], '\n', best[3])

/home/huebscher.m/.local/lib/python3.8/site-packages/imblearn/over_sampling/_smote/base.py:370: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(



S.S.: 0.21, Estimators: 60, f1_score: 0.42
              precision    recall  f1-score   support

         0.0       0.89      0.79      0.84    400666
         1.0       0.35      0.53      0.42     84139

    accuracy                           0.74    484805
   macro avg       0.62      0.66      0.63    484805
weighted avg       0.79      0.74      0.76    484805


S.S.: 0.21, Estimators: 70, f1_score: 0.43
              precision    recall  f1-score   support

         0.0       0.89      0.79      0.84    400666
         1.0       0.35      0.55      0.43     84139

    accuracy                           0.75    484805
   macro avg       0.62      0.67      0.63    484805
weighted avg       0.80      0.75      0.77    484805


S.S.: 0.21, Estimators: 90, f1_score: 0.44
              precision    recall  f1-score   support

         0.0       0.90      0.79      0.84    400666
         1.0       0.36      0.57      0.44     84139

    accuracy                           0.75    484

/home/huebscher.m/.local/lib/python3.8/site-packages/imblearn/over_sampling/_smote/base.py:370: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(



S.S.: 0.23, Estimators: 70, f1_score: 0.45
              precision    recall  f1-score   support

         0.0       0.89      0.77      0.83    400666
         1.0       0.37      0.58      0.45     92153

    accuracy                           0.73    492819
   macro avg       0.63      0.68      0.64    492819
weighted avg       0.79      0.73      0.75    492819


S.S.: 0.23, Estimators: 90, f1_score: 0.46
              precision    recall  f1-score   support

         0.0       0.89      0.77      0.83    400666
         1.0       0.37      0.59      0.46     92153

    accuracy                           0.74    492819
   macro avg       0.63      0.68      0.64    492819
weighted avg       0.79      0.74      0.76    492819



KeyboardInterrupt: 

# Autoencoder - best when dropout is 0

In [43]:
# Hyperparameter tuning - training on non-cancerous patients, w feature engineering
best = (0, 0, 0, None)
# Oversample the minority group to make the data more balanced
for ss in range(5, 35, 10):
    smote = SMOTE(sampling_strategy=ss*0.01, random_state=42)
    X_resampled, y_resampled = smote.fit_resample(skin_cancer_enhanced.drop('target', axis=1), skin_cancer_df['target'])

    # Split the data - training is non-cancerous, test is on all patients to detect anomalies
    X_train = X_resampled[y_resampled == 0]
    X_test = X_resampled

    # Scale the data between 0 and 1
    scaler = MinMaxScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    # Build the autoencoder model - dropout of 5 works best
    for d in range(0, 10, 2):
        
        autoencoder = Sequential([
            Dense(128, input_dim=X_train.shape[1], activation='relu'),
            Dropout(d*0.1),  
            Dense(64, activation='relu'),
            Dropout(d*0.1),
            Dense(32, activation='relu'),
            Dense(64, activation='relu'),
            Dropout(d*0.1),
            Dense(128, activation='relu'),
            Dropout(d*0.1),
            Dense(X_train.shape[1], activation='sigmoid')  # Output layer should match input
        ])

        autoencoder.compile(optimizer='adam', loss='mse')

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        # Train the autoencoder using only the non-cancerous patients
        history = autoencoder.fit(X_train, X_train, epochs=100, batch_size=32, validation_split=0.1,
                                 callbacks=[early_stopping])

        # Find the epoch with the lowest validation loss
        best_epoch = np.argmin(history.history['val_loss']) + 1  # Add 1 since epochs are 1-indexed
        best_val_loss = np.min(history.history['val_loss'])

        # Calculate reconstruction error for each sample
        reconstructed = autoencoder.predict(X_test)
        reconstruction_error = np.mean(np.abs(reconstructed - X_test), axis=1)

        # Threshold the reconstruction error to detect anomalies
        for thresh in range(97, 100):
            threshold = np.percentile(reconstruction_error, thresh)  # Set threshold (e.g., 99th percentile)
            predictions_autoencoder = (reconstruction_error > threshold).astype(int)  # 1 = anomaly (cancer), 0 = normal
            cr = classification_report(y_resampled, predictions_autoencoder)
            f1_score = float(cr.split()[12])
            print(f'\nS.S.: {ss*.01}, Dropout: {d*.1}, Threshold: {thresh}, Best Epoch {best_epoch}',
                  f'f1 score: {f1_score}\n', cr)
            if f1_score > best[2]:
                best = (ss, d, thresh, best_epoch, f1_score, cr)

print('Best:')
print(best[:4])
print(best[5])

Epoch 1/100
11269/11269 [==============================] - 22s 2ms/step - loss: 0.0022 - val_loss: 2.5900e-04
Epoch 2/100
11269/11269 [==============================] - 21s 2ms/step - loss: 2.0804e-04 - val_loss: 2.8170e-04
Epoch 3/100
11269/11269 [==============================] - 22s 2ms/step - loss: 8.6720e-04 - val_loss: 1.1256e-04
Epoch 4/100
11269/11269 [==============================] - 22s 2ms/step - loss: 1.3901e-04 - val_loss: 1.0572e-04
Epoch 5/100
11269/11269 [==============================] - 22s 2ms/step - loss: 1.3141e-04 - val_loss: 6.2773e-05
Epoch 6/100
11269/11269 [==============================] - 22s 2ms/step - loss: 0.0027 - val_loss: 5.5272e-05
Epoch 7/100
11269/11269 [==============================] - 22s 2ms/step - loss: 1.2240e-04 - val_loss: 4.8739e-05
Epoch 8/100
11269/11269 [==============================] - 23s 2ms/step - loss: 2.5014e-04 - val_loss: 5.3878e-05
Epoch 9/100
11269/11269 [==============================] - 22s 2ms/step - loss: 1.7224e-04 - val

KeyboardInterrupt: 